# SC1015 Assignment

## Table of Contents
---
1. Importing the Data
    - Why we are only using 2018 and 2019 data
    - Importing the actual data 
2. Data Cleaning
    - Date and Time
3. EDA
---
## Problem Statement
How do we predict a flight delay

### Motivation
Returning from covid

### Dataset
The dataset includes 2018 to 2020
However, only 2018 and 2019 is used as explained more in-depth
---
## Import Libraries


In [112]:
#imports
import datetime
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sb

pd.set_option("display.max_columns", 500)
plt.style.use('ggplot')

## Importing the data

In [57]:
# Mount from google colab
# from google.colab import drive
# drive.mount('/content/drive')
# root = "/content/drive/MyDrive/Datasets/SC1015/"

# Personal laptops
root = "../data/"

### Importing the data to be used

In [81]:
# Import the data that will actually be used
column_subset = ['FlightDate',
                 'Airline',
                 'Operating_Airline',
                 'Origin',
                 'Dest',
                 'Cancelled',
                 'Diverted',
                 'CRSDepTime',
                 'DepTime',
                 'ArrDelayMinutes', 
                 'DepDelayMinutes', 
                 'OriginAirportID',
                 'OriginStateName',
                 'DestAirportID',
                 'DestCityName',
                 'DestStateName',
                 'ArrTime', 'CRSArrTime'
                 #'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay'
                ]

years = [2018, 2019]
dataframes = []

for year in years:
    df = pd.read_parquet(f"{root}Combined_Flights_{year}.parquet", columns=column_subset)
    dataframes.append(df)
del df

flights_df = pd.concat(dataframes).reset_index(drop=True)

In [71]:
print(flights_df.shape)

(13781196, 16)


In [72]:
print(flights_df.columns)

Index(['FlightDate', 'Airline', 'Operating_Airline', 'Origin', 'Dest',
       'Cancelled', 'Diverted', 'CRSDepTime', 'DepTime', 'ArrDelayMinutes',
       'DepDelayMinutes', 'OriginAirportID', 'OriginStateName',
       'DestAirportID', 'DestCityName', 'DestStateName'],
      dtype='object')


In [73]:
# Airports
airports_df = pd.read_csv(f"{root}airlines.csv")

## Data Cleaning

### Categorising appropriate columns

In [13]:
flights_df.columns

Index(['FlightDate', 'Airline', 'Origin', 'Dest', 'Cancelled', 'Diverted',
       'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'DepDelay', 'ArrTime',
       'ArrDelayMinutes', 'AirTime', 'CRSElapsedTime', 'ActualElapsedTime',
       'Distance', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
       'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline',
       'Flight_Number_Marketing_Airline', 'Operating_Airline',
       'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline',
       'Tail_Number', 'Flight_Number_Operating_Airline', 'OriginAirportID',
       'OriginAirportSeqID', 'OriginCityMarketID', 'OriginCityName',
       'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac',
       'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'DestCityName',
       'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'DepDel15',
       'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOu

In [74]:
flights_df.describe() 

CRSDepTime       DepTime  ArrDelayMinutes  DepDelayMinutes  \
count  1.378120e+07  1.354792e+07     1.350388e+07     1.354670e+07   
mean   1.330309e+03  1.334702e+03     1.411466e+01     1.394830e+01   
std    4.909026e+02  5.047816e+02     4.665277e+01     4.687052e+01   
min    1.000000e+00  1.000000e+00     0.000000e+00     0.000000e+00   
25%    9.150000e+02  9.160000e+02     0.000000e+00     0.000000e+00   
50%    1.323000e+03  1.327000e+03     0.000000e+00     0.000000e+00   
75%    1.735000e+03  1.745000e+03     8.000000e+00     7.000000e+00   
max    2.359000e+03  2.400000e+03     2.973000e+03     2.976000e+03   

       OriginAirportID  DestAirportID  
count     1.378120e+07   1.378120e+07  
mean      1.269441e+04   1.269436e+04  
std       1.516753e+03   1.516743e+03  
min       1.013500e+04   1.013500e+04  
25%       1.129200e+04   1.129200e+04  
50%       1.288900e+04   1.288900e+04  
75%       1.405700e+04   1.405700e+04  
max       1.686900e+04   1.686900e+04

In [75]:
flights_df.dtypes

FlightDate           datetime64[ns]
Airline                      object
Operating_Airline            object
Origin                       object
Dest                         object
Cancelled                      bool
Diverted                       bool
CRSDepTime                    int64
DepTime                     float64
ArrDelayMinutes             float64
DepDelayMinutes             float64
OriginAirportID               int64
OriginStateName              object
DestAirportID                 int64
DestCityName                 object
DestStateName                object
dtype: object

In [76]:
cat_cols = ["Airline", "Operating_Airline" , "Origin", "Dest", "OriginStateName", "DestStateName"]
for c in cat_cols:
    flights_df[c] = flights_df[c].astype("category")

adding a new column to categorise flights as either delayed, cancelled or on time.  
we have defined them as follows:  
Cancelled: if the Cancelled column value == 1  
Delayed: if the ArrDelayMinutes column value is > 0  
we have assumed a flight to be delayed if the arrival time is after the estimated arrival time

In [90]:
flights_df['DelaySituation'] = 'OnTime'
flights_df.loc[flights_df['Cancelled'], 'DelaySituation'] = 'Cancelled'
flights_df.loc[(flights_df['ArrDelayMinutes'] > 0) & (~flights_df['Cancelled']), 'DelaySituation'] = 'Delayed'
display(flights_df)

FlightDate                   Airline Operating_Airline Origin Dest  \
0        2018-01-23         Endeavor Air Inc.                9E    ABY  ATL   
1        2018-01-24         Endeavor Air Inc.                9E    ABY  ATL   
2        2018-01-25         Endeavor Air Inc.                9E    ABY  ATL   
3        2018-01-26         Endeavor Air Inc.                9E    ABY  ATL   
4        2018-01-27         Endeavor Air Inc.                9E    ABY  ATL   
...             ...                       ...               ...    ...  ...   
13781191 2019-01-23  ExpressJet Airlines Inc.                EV    MEM  IAH   
13781192 2019-01-24  ExpressJet Airlines Inc.                EV    MEM  IAH   
13781193 2019-01-25  ExpressJet Airlines Inc.                EV    MEM  IAH   
13781194 2019-01-26  ExpressJet Airlines Inc.                EV    MEM  IAH   
13781195 2019-01-28  ExpressJet Airlines Inc.                EV    MEM  IAH   

          Cancelled  Diverted CRSDepTime DepTime  ArrDelayMinutes  \
0             False     False       1202    1157              0.0   
1             False     False       1202    1157              0.0   
2             False     False       1202    1153              0.0   
3             False     False       1202    1150              0.0   
4             False     False       1400    1355              0.0   
...             ...       ...        ...     ...              ...   
13781191      False     False        640     634             13.0   
13781192      False     False        640     631              0.0   
13781193      False     False        640     632              0.0   
13781194      False     False        640     630              0.0   
13781195      False     False        640     632              0.0   

          DepDelayMinutes  OriginAirportID OriginStateName  DestAirportID  \
0                     0.0            10146         Georgia          10397   
1                     0.0            10146         Georgia          10397   
2                     0.0            10146         Georgia          10397   
3                     0.0            10146         Georgia          10397   
4                     0.0            10146         Georgia          10397   
...                   ...              ...             ...            ...   
13781191              0.0            13244       Tennessee          12266   
13781192              0.0            13244       Tennessee          12266   
13781193              0.0            13244       Tennessee          12266   
13781194              0.0            13244       Tennessee          12266   
13781195              0.0            13244       Tennessee          12266   

         DestCityName DestStateName ArrTime CRSArrTime    CRSDepFlightDate  \
0         Atlanta, GA       Georgia    1256       1304 2018-01-23 12:02:00   
1         Atlanta, GA       Georgia    1258       1304 2018-01-24 12:02:00   
2         Atlanta, GA       Georgia    1302       1304 2018-01-25 12:02:00   
3         Atlanta, GA       Georgia    1253       1304 2018-01-26 12:02:00   
4         Atlanta, GA       Georgia    1459       1500 2018-01-27 14:00:00   
...               ...           ...     ...        ...                 ...   
13781191  Houston, TX         Texas     853        840 2019-01-23 06:40:00   
13781192  Houston, TX         Texas     830        840 2019-01-24 06:40:00   
13781193  Houston, TX         Texas     828        840 2019-01-25 06:40:00   
13781194  Houston, TX         Texas     831        840 2019-01-26 06:40:00   
13781195  Houston, TX         Texas     825        840 2019-01-28 06:40:00   

         DelaySituation  
0                OnTime  
1                OnTime  
2                OnTime  
3                OnTime  
4                OnTime  
...                 ...  
13781191        Delayed  
13781192         OnTime  
13781193         OnTime  
13781194         OnTime  
13781195         OnTime  

[13781196 rows x 20 columns]

### Creating a new column to get the expected Departure and Arrival times

In [82]:
time_cols = ["ArrTime", "DepTime", "CRSArrTime", "CRSDepTime"]
for col in time_cols:
    flights_df[col] = flights_df[col].fillna(0)
    flights_df[col] = pd.to_numeric(flights_df[col], errors='coerce')
    flights_df[col] = flights_df[col].astype(int)
    flights_df[col] = flights_df[col].astype(str)

In [83]:
def combine_date_time(date_col, time_col, new_col_name):
    date_str = flights_df[date_col].astype(str)
    time_str = flights_df[time_col].astype(str)
    time_str = time_str.apply(lambda x: x.zfill(4) if len(x) < 4 else x.rjust(4, '0'))
    datetime_str = date_str + ' ' + time_str
    
    flights_df[new_col_name] = pd.to_datetime(datetime_str, format='%Y-%m-%d %H%M')


In [84]:
#flights_df.loc[:5, ["CarrierDelay", "WeatherDelay", "NASDelay", "SecurityDelay", "LateAircraftDelay"]]

In [85]:
combine_date_time("FlightDate", "CRSDepTime", "CRSDepFlightDate")

In [86]:
flights_df.loc[:10, ["CRSDepFlightDate", "CRSDepTime", "DepTime", "DepDelayMinutes", "CRSArrTime", "ArrTime", "ArrDelayMinutes"]]

CRSDepFlightDate CRSDepTime DepTime  DepDelayMinutes CRSArrTime ArrTime  \
0  2018-01-23 12:02:00       1202    1157              0.0       1304    1256   
1  2018-01-24 12:02:00       1202    1157              0.0       1304    1258   
2  2018-01-25 12:02:00       1202    1153              0.0       1304    1302   
3  2018-01-26 12:02:00       1202    1150              0.0       1304    1253   
4  2018-01-27 14:00:00       1400    1355              0.0       1500    1459   
5  2018-01-28 12:02:00       1202    1202              NaN       1304    1326   
6  2018-01-29 12:02:00       1202    1204              2.0       1304    1303   
7  2018-01-30 12:02:00       1202    1153              0.0       1304    1255   
8  2018-01-31 12:02:00       1202    1153              0.0       1304    1304   
9  2018-01-03 10:37:00       1037    1101             24.0       1137    1159   
10 2018-01-04 10:37:00       1037    1032              0.0       1137    1125   

    ArrDelayMinutes  
0               0.0  
1               0.0  
2               0.0  
3               0.0  
4               0.0  
5              22.0  
6               0.0  
7               0.0  
8               NaN  
9              22.0  
10              0.0

### Checking how complete the dataset is


In [91]:
missing_df = flights_df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(flights_df.shape[0]-missing_df['missing values'])/flights_df.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)

variable  missing values  filling factor (%)
0     ArrDelayMinutes          277313           97.987744
1     DepDelayMinutes          234493           98.298457
2          CRSArrTime               0          100.000000
3             ArrTime               0          100.000000
4       DestStateName               0          100.000000
5        DestCityName               0          100.000000
6       DestAirportID               0          100.000000
7     OriginStateName               0          100.000000
8     OriginAirportID               0          100.000000
9          FlightDate               0          100.000000
10            DepTime               0          100.000000
11         CRSDepTime               0          100.000000
12           Diverted               0          100.000000
13          Cancelled               0          100.000000
14               Dest               0          100.000000
15             Origin               0          100.000000
16  Operating_Airline               0          100.000000
17            Airline               0          100.000000
18   CRSDepFlightDate               0          100.000000
19     DelaySituation               0          100.000000

## EDA

### Which airline has the most delays

In [15]:
flights_df['Airline'].value_counts()

Southwest Airlines Co.                       2716498
Delta Air Lines Inc.                         1453704
SkyWest Airlines Inc.                        1368547
American Airlines Inc.                       1331959
United Air Lines Inc.                        1247475
JetBlue Airways                               602421
Republic Airlines                             533125
Envoy Air                                     454834
Alaska Airlines Inc.                          449568
Comair Inc.                                   406121
Mesa Airlines Inc.                            381528
Spirit Air Lines                              381023
Endeavor Air Inc.                             377276
ExpressJet Airlines Inc.                      301457
Frontier Airlines Inc.                        255578
Horizon Air                                   207142
Air Wisconsin Airlines Corp                   205657
Allegiant Air                                 201526
Hawaiian Airlines Inc.                        

Creating a dataframe of the airlines and sorting the number of delayed, ontime and cancelled flights.  
then dividing those numbers by number of total flights per airline and multplying by 100 so as to get percentages

In [115]:
flights_df['Airline'] = flights_df['Airline'].astype('category')
airline_df = flights_df.groupby('Airline')['DelaySituation'].value_counts().unstack()


total_flights = airline_df.sum(axis=1)
print(type(total_flights))
for value in airline_df.columns:
    airline_df[value] = (airline_df[value] / total_flights)*100

total_flights = total_flights.sort_values(ascending=False).index
display(airline_df)


<class 'pandas.core.series.Series'>


DelaySituation                             Cancelled    Delayed     OnTime
Airline                                                                   
Air Wisconsin Airlines Corp                 3.270008  31.082822  65.647170
Alaska Airlines Inc.                        1.041222  34.724224  64.234554
Allegiant Air                               0.691226  38.062086  61.246688
American Airlines Inc.                      1.920630  37.410911  60.668459
Cape Air                                    0.120409  26.189043  73.690548
Capital Cargo International                 4.281944  33.003570  62.714485
Comair Inc.                                 2.484727  35.077477  62.437796
Commutair Aka Champlain Enterprises, Inc.   5.162075  45.074579  49.763346
Compass Airlines                            0.570065  33.693488  65.736447
Delta Air Lines Inc.                        0.274678  27.856703  71.868620
Empire Airlines Inc.                        5.301339  36.277902  58.420759
Endeavor Air Inc.                           1.712274  28.047371  70.240355
Envoy Air                                   3.375297  35.732157  60.892545
ExpressJet Airlines Inc.                    3.430340  36.313637  60.256023
Frontier Airlines Inc.                      1.803363  41.159646  57.036991
GoJet Airlines, LLC d/b/a United Express    1.657536  32.084511  66.257953
Hawaiian Airlines Inc.                      0.230888  33.436348  66.332765
Horizon Air                                 1.726352  35.540354  62.733294
JetBlue Airways                             1.690346  39.496465  58.813189
Mesa Airlines Inc.                          2.640960  36.034839  61.324202
Peninsula Airways Inc.                     15.486885  48.544736  35.968379
Republic Airlines                           2.500539  32.218148  65.281313
SkyWest Airlines Inc.                       1.833770  32.708705  65.457525
Southwest Airlines Co.                      1.910438  35.715653  62.373909
Spirit Air Lines                            1.398078  32.224301  66.377620
Trans States Airlines                       3.496518  39.323932  57.179550
United Air Lines Inc.                       0.824626  35.120944  64.054430
Virgin America                              2.450481  35.868704  61.680815

ImportError: cannot import name 'AxisInt' from 'pandas._typing' (/Users/mythilimulani/opt/anaconda3/lib/python3.9/site-packages/pandas/_typing.py)

In [117]:
airline_df.loc[total_flights].sort_values('OnTime', ascending=False) \
    [['OnTime','Delayed','Cancelled']] \
    .style.background_gradient(axis=Axis(0),cmap='YlOrRd')

ImportError: cannot import name 'AxisInt' from 'pandas._typing' (/Users/mythilimulani/opt/anaconda3/lib/python3.9/site-packages/pandas/_typing.py)

Plotting airline vs number of flights 

### Taking a look at the delay time distributions

Based on: https://en.wikipedia.org/wiki/Flight_cancellation_and_delay
Flight delays can be categorised into 3 different categories.

We'll start by plotting for the frequency of delays of those 60 minutes or under.

In [25]:
flights_df.query('DepDelayMinutes > 0 & DepDelayMinutes <= 60')['DepDelayMinutes'].plot(kind='hist', bins=30)

<Axes: ylabel='Frequency'>